In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.signal import savgol_filter

import data_cleaning
from modelling_fctns import double_logistic, normalized_difference

In [2]:
def determine_phenological_stages(ndvi, doy, a=0.10):
    # Step 2: Determine start, end, and mid-point of the season

    # Determine NDVI threshold
    ndvi_max = np.max(ndvi)
    ndvi_base = np.min(ndvi[ndvi > 0])
    ndvi_threshold = a * (ndvi_max - ndvi_base) + ndvi_base

    # Determine start, end, and mid-point of the season
    doy_start = doy[np.argmax(ndvi > ndvi_threshold)]
    doy_end = doy[-(np.argmax(ndvi[::-1] > ndvi_threshold)+1)]  # Search in reverse for last occurrence
    doy_midpoint = int(0.5 * (doy_start + doy_end) + doy_start)

    return ndvi_threshold, doy_start, doy_end, doy_midpoint

def calculate_small_integral(ndvi, doy, doy_midpoint, n=100):
    # Step 3: Calculate the small integral (summed NDVI, sNDVI)
    # Determine crop greenup date
    doy_greenup = doy_midpoint - n

    # Calculate the small integral (sNDVI)
    ndvi_base = np.min(ndvi[ndvi > 0])
    s_ndvi = (ndvi[doy >= doy_greenup] - ndvi_base)

    return s_ndvi

def get_biomass(df_ndvi, plotting=False):
    # Convert date to datetime
    df_ndvi['date'] = pd.to_datetime(df_ndvi['date'])
    #sort by date
    df_ndvi = df_ndvi.sort_values('date')
    df_ndvi = df_ndvi.loc[df_ndvi['NDVI'] != -9999]
    #groupby day and mean
    df_ndvi = df_ndvi[['NDVI','date', 'cs', 'cs_cdf']].groupby('date').mean().reset_index()
    #mask out if cs>0.55
    df_ndvi = df_ndvi.loc[df_ndvi['cs'] > 0.55]

    #get start and end date, truncate time
    start_year = df_ndvi['date'].dt.year.values[0]
    start_dt = df_ndvi['date'].dt.normalize().values[0]
    end_dt = df_ndvi['date'].dt.normalize().values[-1]
    #get daily timerange from start to end
    date_range = pd.date_range(start=start_dt, end=end_dt, freq='D')

    #interpolate NDVI for every day
    df_ndvi = df_ndvi.set_index('date')
    df_ndvi = df_ndvi[['NDVI','cs', 'cs_cdf']].resample('D').mean()
    df_ndvi = df_ndvi.interpolate(method='linear')
    df_ndvi = df_ndvi.reset_index()

    doy = df_ndvi['date'].dt.dayofyear.values #np.array([i for i in range(1, 366)])
    first_doy = doy[0]
    days_elapsed = np.array(range(len(doy)))

    # Step 1: Interpolate the time series
    ndvi_interp = df_ndvi['NDVI'].values.copy()
    cs_interp = df_ndvi['cs'].values.copy()

    # Check for NaNs and negative numbers in NDVI time series and return nans if either condition is met - SAJchange
    has_nan = any(np.isnan(x) for x in ndvi_interp if isinstance(x, (int, float)))
    #has_negative = any(x < 0 for x in ndvi_interp if isinstance(x, (int, float)))
    #if has_nan or has_negative:
    if has_nan:
        print("Error: The list contains NaN values.")
        s_ndvi_crop = date_range = np.nan # = ndvi_interp sajchange - addition of ndvi_interp here
        return np.cumsum(s_ndvi_crop), date_range#, ndvi_interp

    #interpolate_ndvi(ndvi_values, doy)
    # smooth the time series with savitzky-golay filter
    ndvi_interp = savgol_filter(ndvi_interp, 51, 2)

    # Step 2: Determine start, end, and mid-point of the season
    ndvi_threshold, doy_start, doy_end, doy_midpoint = determine_phenological_stages(ndvi_interp, days_elapsed)

    # Step 3: Calculate the small integral (summed NDVI, sNDVI)
    n = 100
    s_ndvi = calculate_small_integral(ndvi_interp, days_elapsed, doy_midpoint, n)

    #set start_dt to be greenup which is midpoint-n days
    start_dt = start_dt + pd.Timedelta(days=doy_midpoint) - pd.Timedelta(days=n)

    #if start_dt earlier than start of date range, backfill sndvi
    if start_dt <= date_range[0]:
        s_ndvi_crop = np.zeros(len(pd.date_range(start=start_dt, end=end_dt, freq='D')))
        num_bfill_days = len(s_ndvi_crop) - len(s_ndvi)
        if len(pd.date_range(start=start_dt, end=end_dt, freq='D')) < len(date_range):
            s_ndvi_crop[num_bfill_days:] = s_ndvi[:len(pd.date_range(start=start_dt, end=end_dt, freq='D'))]
        s_ndvi_crop[num_bfill_days:] = s_ndvi
        date_range = pd.date_range(start=start_dt, end=end_dt, freq='D')
    else:
        s_ndvi_crop = s_ndvi
        date_range = pd.date_range(start=start_dt, end=end_dt, freq='D')

    #1% chance of plotting == 1 else 0
    if plotting:
        plt.figure(figsize=(12,6))

        # Visualization (optional)
        plt.plot(days_elapsed+first_doy, ndvi_values, label='Original NDVI')
        plt.plot(days_elapsed+first_doy, ndvi_interp, label='Interpolated NDVI')
        plt.plot(days_elapsed+first_doy, cs_interp, 'r*', alpha=.25, label='Cloud Score')
        plt.axhline(y=ndvi_threshold, color='r', linestyle='--', label='NDVI Threshold')
        plt.axvline(x=doy_start+first_doy, color='g', linestyle='--', label='Start of Season')
        plt.axvline(x=doy_end+first_doy, color='b', linestyle='--', label='End of Season')
        plt.axvline(x=doy_midpoint+first_doy, color='m', linestyle='--', label='Midpoint of Season')
        plt.title('NDVI Time Series and Phenological Stages')
        plt.xlabel('Day of Year')
        plt.ylabel('NDVI')
        plt.legend()
        plt.show()

        # Print results
        print(f'NDVI Threshold: {ndvi_threshold}')
        print(f'Start of Season (DOY): {doy_start+first_doy}')
        print(f'End of Season (DOY): {doy_end+first_doy}')
        print(f'Midpoint of Season (DOY): {doy_midpoint+first_doy}')
        print(f'Small Integral (sNDVI): {np.sum(s_ndvi)}')

    assert len(date_range) == len(s_ndvi_crop), f'date range and s_ndvi length mismatch, {len(date_range)} != {len(s_ndvi_crop)}'
    return np.cumsum(s_ndvi_crop), date_range

In [2]:
ds2 = pd.read_csv('C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\earth_engine_MP\\Saved_files\\MODIS\\Germany\\satdata0.csv')
ds2 = data_cleaning.add_SOS_to_df(ds2)
ds2 = data_cleaning.add_EOS_to_df(ds2)
ds2['NDVI'] = normalized_difference(ds2['median sur_refl_b02'], ds2['median sur_refl_b01'])
max_value_interpolated = data_cleaning.map_max_value_int(ds2, window_size=8)
array_input = data_cleaning.make_df_samples(max_value_interpolated, sample_number = 1000, m_window_size = 5)
sampled_locs_input = data_cleaning.make_tensor_from_timeseries(max_value_interpolated, sample_number = 100, m_window_size = 5, format_choice = 'numpy')

In [3]:
ds2

,Unnamed: 0,Time,lat,lon,Stations_Id,median sur_refl_b01,median sur_refl_b02,median sur_refl_b03,median sur_refl_b04,formatted_time,SOS,EOS,NDVI
0,0,1577836800000,54.7833,9.4333,7501,1080.0,1386.0,1118.0,1089.0,2020-01-01 00:00:00+00:00,103.0,297.0,0.124088
1,3,1578096000000,54.7833,9.4333,7501,502.0,707.0,589.0,537.0,2020-01-04 00:00:00+00:00,103.0,297.0,0.169562
2,18,1579392000000,54.7833,9.4333,7501,827.0,1384.0,542.0,786.0,2020-01-19 00:00:00+00:00,103.0,297.0,0.251922
3,35,1580860800000,54.7833,9.4333,7501,546.0,1028.0,353.0,525.0,2020-02-05 00:00:00+00:00,103.0,297.0,0.306226
4,58,1582848000000,54.7833,9.4333,7501,146.0,410.0,83.0,68.0,2020-02-28 00:00:00+00:00,103.0,297.0,0.474820
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15341,1054,1669939200000,54.8000,9.6333,7569,2542.0,2068.0,3783.0,3158.0,2022-12-02 00:00:00+00:00,104.0,299.0,-0.102820
15342,1063,1670716800000,54.8000,9.6333,7569,1683.0,3394.0,1963.0,1900.0,2022-12-11 00:00:00+00:00,104.0,299.0,0.337010
15343,1068,1671148800000,54.8000,9.6333,7569,3458.0,4384.0,4232.0,3730.0,2022-12-16 00:00:00+00:00,104.0,299.0,0.118082
15344,1075,1671753600000,54.8000,9.6333,7569,533.0,2399.0,608.0,705.0,2022-12-23 00:00:00+00:00,104.0,299.0,0.636426


In [7]:
df_time_adjust = ds2.reset_index().rename(columns={'formatted_time': 'actual_time'}).reset_index()
df_time_adjust['time from edge'] = (df_time_adjust['actual_time'] - df_time_adjust['formatted_time']).dt.days

KeyError: 'formatted_time'